In [1]:
!pip install git+git://github.com/openvax/mhc2flurry.git#egg=mhc2flurry
!mhc2flurry-downloads fetch data_iedb data_antigens
!pip -q install transformers seqeval


****************************************
The requested download 'data_iedb' has already been downloaded. To re-download this data, first run: 
	rm -rf /root/.local/share/mhc2flurry/1/0.0.1/data_iedb/
in a shell and then re-run this command.
****************************************
****************************************
The requested download 'data_antigens' has already been downloaded. To re-download this data, first run: 
	rm -rf /root/.local/share/mhc2flurry/1/0.0.1/data_antigens/
in a shell and then re-run this command.
****************************************
Fetching 0/7 downloads from release 0.0.1
DOWNLOAD NAME                             ALREADY DOWNLOADED?    WILL DOWNLOAD NOW?    URL                  
allele_sequences                          NO                     NO                    https://github.com/openvax/mhc2flurry/releases/download/pre-0.0.1/allele_sequences.20210222.tar.bz2 
data_curated                              NO                     NO                    ht

In [2]:
!nvidia-smi

Fri Apr  9 21:11:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy
import pandas
import seaborn
import logging
import time
import collections
import os
from os import environ
from matplotlib import pyplot

import sklearn
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.linear_model

%matplotlib inline
logging.basicConfig(level="DEBUG")
pandas.set_option('display.max_columns', 60)

from copy import deepcopy
import shutil
import bz2

import tensorflow as tf

def ppv(y_true, predictions):
    df = pandas.DataFrame({"prediction": predictions, "y_true": y_true})
    return df.sort_values("prediction", ascending=False)[:int(y_true.sum())].y_true.mean()

import json
from scipy.stats import pearsonr

import tqdm

from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

import mhc2flurry
from mhc2flurry.downloads import get_path
import mhc2flurry.allele_encoding_pair
import mhc2flurry.allele_encoding
import mhc2flurry.fasta
import mhc2flurry.common
import mhc2flurry.encodable_sequences

from mhc2flurry.regression_target import from_ic50, to_ic50

import tensorflow as tf ; print("GPU AVAILABLE" if tf.test.is_gpu_available() else "GPU NOT AVAILABLE")

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:root:Configured MHC2FLURRY_DOWNLOADS_DIR: /root/.local/share/mhc2flurry/1/0.0.1
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU AVAILABLE


In [4]:
#!conda install --yes pytorch

In [5]:

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForTokenClassification, BertTokenizerFast, EvalPrediction
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
import re

In [6]:
!hostname
!ls "$(mhc2flurry-downloads path data_iedb)"

9196aad9e03a
DOWNLOAD_INFO.csv  LOG.txt.bz2		    tcell_full_v3.csv.bz2
GENERATE.sh	   mhc_ligand_full.csv.bz2  viruses.uniprot.fasta.gz


In [7]:
epitopes_df = pandas.read_csv(get_path("data_iedb", "tcell_full_v3.csv.bz2"), skiprows=1)
epitopes_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,14,23,24,25,26,29,30,31,32,33,34,35,36,37,38,41,42,43,45,46,47,53,54,55,56,57,62,63,64,65,66,67,68,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,98,99,100,103,106,116,118,124,125,126,127,133,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Assay IRI,Reference IRI,Type,PubMed ID,Authors,Journal,Date,Title,Submission ID,Epitope IRI,Object Type,Description,Starting Position,Ending Position,Non-peptidic epitope IRI,Antigen Name,Antigen IRI,Parent Protein,Parent Protein IRI,Organism Name,Organism IRI,Parent Species,Parent Species IRI,Epitope Comments,Epitope Relationship,Object Type.1,Description.1,Starting Position.1,Ending Position.1,Non-peptidic object IRI,...,Cell Type,Effector Cell Type IRI,Cell Culture Conditions,Assayed TCR Molecule Name,TCR IRI,PDB ID,Cell Tissue Type.1,APC Cell Tissue Type IRI,Cell Type.1,APC Cell Type IRI,Cell Culture Conditions.1,Allele Name,Class,Allele IRI,Allele Evidence Code,Antigen Epitope Relation,Antigen Object Type,Antigen Description,Starting Position.2,Ending Position.2,Non-peptidic Antigen IRI,Antigen Source Molecule Name,Antigen Source Molecule IRI,Protein Parent Name.1,Protein Parent IRI.1,Antigen Organism Name,Antigen Organism IRI,Organism Species Name,Organism Species IRI,Assay Comments
0,http://www.iedb.org/assay/29,http://www.iedb.org/reference/274,Literature,15448372.0,Yi-Hsiang Huang; Mi-Hua Tao; Cheng-Po Hu; Wan-...,J Gen Virol,2004,Identification of novel HLA-A*0201-restricted ...,NaN,http://www.iedb.org/epitope/31803,Linear peptide,KLEDLERDL,26.0,34.0,NaN,large delta antigen,http://www.ncbi.nlm.nih.gov/protein/AAG26087.1,Large delta antigen,http://www.uniprot.org/uniprot/P29996,Hepatitis delta virus TW2667,https://ontology.iedb.org/ontology/ONTIE_0000432,Hepatitis delta virus,http://purl.obolibrary.org/obo/NCBITaxon_12475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Splenocyte,http://purl.obolibrary.org/obo/CL_2000074,Direct Ex Vivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HLA-A*02:01,I,http://purl.obolibrary.org/obo/MRO_0001007,MHC binding assay,Epitope,Linear peptide,KLEDLERDL,26.0,34.0,NaN,large delta antigen,http://www.ncbi.nlm.nih.gov/protein/AAG26087.1,Large delta antigen,http://www.uniprot.org/uniprot/P29996,Hepatitis delta virus TW2667,http://iedb.org/taxon/10000523,Hepatitis delta virus,http://purl.obolibrary.org/obo/NCBITaxon_12475,NaN
1,http://www.iedb.org/assay/37,http://www.iedb.org/reference/281,Literature,7684468.0,V J La Posta; D D Auperin; R Kamin-Lewis; G A ...,J Virol,1993,Cross-protection against lymphocytic choriomen...,NaN,http://www.iedb.org/epitope/25946,Linear peptide,IEQQADNMITEMLQK,403.0,417.0,NaN,Glycoprotein polyprotein,http://www.ncbi.nlm.nih.gov/protein/P08669.1,Pre-glycoprotein polyprotein GP complex,http://www.uniprot.org/uniprot/P08669,Lassa mammarenavirus,http://purl.obolibrary.org/obo/NCBITaxon_11620,Lassa mammarenavirus,http://purl.obolibrary.org/obo/NCBITaxon_11620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T cell CD4+,http://purl.obolibrary.org/obo/CL_0000624,Direct Ex Vivo,NaN,NaN,NaN,Spleen,http://purl.obolibrary.org/obo/UBERON_0002106,Splenocyte,http://purl.obolibrary.org/obo/CL_2000074,Direct Ex Vivo,H2-IAk,II,http://purl.obolibrary.org/obo/MRO_0000976,Not determined,Epitope,Linear peptide,IEQQADNMITEMLQK,403.0,417.0,NaN,Glycoprotein polyprotein,http://www.ncbi.nlm.nih.gov/protein/P08669.1,Pre-glycoprotein polyprotein GP complex,http://www.uniprot.org/uniprot/P08669,Lassa mammarenavirus,http://purl.obolibrary.org/obo/NCBITaxon_11620,Lassa mammarenavirus,http://purl.obolibrary.org/obo/NCBITaxon_11620,CD4+ T cells from the spleen of mice infected ...
2,http://www.iedb.org/assay/42,http://www.iedb.org/reference/283,Literature,15634878.0,Evan R Jellison; Sung-Kwon Kim; Raymond M Welsh,J Immunol,2005,Cutting edge: MHC class II-restricted killing ...,NaN,http://www.iedb.org/epitope/21045,Linear peptide,GLNGPDIYKGVYQFKSVEFD,61.0,80.0,NaN,Glycoprotein polyprotein,http://www.ncbi.nlm.nih.gov/protein/P07399.1,Pre-glycoprotein polyprotein GP complex,http://www.uniprot.org/uniprot/P09991,Lymphocytic choriomeningitis virus (strain Arm...,http://purl.obolibrary.org/obo/NCBITaxon_11624,Lymphocytic choriomeningitis mammarenavirus,http://purl.obolibrary.org/obo/NCBITaxon_11623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Splenocyte,http

In [8]:
epitopes_df.columns.tolist()

['Assay IRI',
 'Reference IRI',
 'Type',
 'PubMed ID',
 'Authors',
 'Journal',
 'Date',
 'Title',
 'Submission ID',
 'Epitope IRI',
 'Object Type',
 'Description',
 'Starting Position',
 'Ending Position',
 'Non-peptidic epitope IRI',
 'Antigen Name',
 'Antigen IRI',
 'Parent Protein',
 'Parent Protein IRI',
 'Organism Name',
 'Organism IRI',
 'Parent Species',
 'Parent Species IRI',
 'Epitope Comments',
 'Epitope Relationship',
 'Object Type.1',
 'Description.1',
 'Starting Position.1',
 'Ending Position.1',
 'Non-peptidic object IRI',
 'Synonyms',
 'Antigen Name.1',
 'Antigen IRI.1',
 'Parent Protein.1',
 'Parent Protein IRI.1',
 'Organism Name.1',
 'Organism IRI.1',
 'Parent Organism',
 'Parent Organism IRI',
 'Name',
 'Host IRI',
 'Geolocation',
 'Geolocation IRI',
 'MHC Types Present',
 'Process Type',
 'Disease State',
 'Disease IRI',
 'Disease Stage',
 'Immunogen Epitope Relation',
 'Immunogen Object Type',
 'Immunogen Description',
 'Immunogen Starting Position',
 'Immunogen En

In [9]:
epitopes_df["Process Type"].value_counts()

Administration in vivo                                                             157536
Occurrence of infectious disease                                                    64821
Occurrence of allergy                                                               44878
Environmental exposure to endemic/ubiquitous agent without evidence for disease     35029
No immunization                                                                     29567
Exposure with existing immune reactivity without evidence for disease               27254
Occurrence of autoimmune disease                                                    14020
Occurrence of cancer                                                                 7388
Occurrence of disease                                                                4465
Administration in vivo to cause disease                                              3103
Exposure without evidence for disease                                                2905
Administra

In [10]:
use_epitopes_df = epitopes_df.loc[
    (epitopes_df["Process Type"] == "Occurrence of infectious disease") &
    (epitopes_df["Class"] == "II") &
    (epitopes_df["Number of Subjects Responded"] >= 2)
].sort_values("Number of Subjects Responded", ascending=False).drop_duplicates("Description")
use_epitopes_df

,Assay IRI,Reference IRI,Type,PubMed ID,Authors,Journal,Date,Title,Submission ID,Epitope IRI,Object Type,Description,Starting Position,Ending Position,Non-peptidic epitope IRI,Antigen Name,Antigen IRI,Parent Protein,Parent Protein IRI,Organism Name,Organism IRI,Parent Species,Parent Species IRI,Epitope Comments,Epitope Relationship,Object Type.1,Description.1,Starting Position.1,Ending Position.1,Non-peptidic object IRI,...,Cell Type,Effector Cell Type IRI,Cell Culture Conditions,Assayed TCR Molecule Name,TCR IRI,PDB ID,Cell Tissue Type.1,APC Cell Tissue Type IRI,Cell Type.1,APC Cell Type IRI,Cell Culture Conditions.1,Allele Name,Class,Allele IRI,Allele Evidence Code,Antigen Epitope Relation,Antigen Object Type,Antigen Description,Starting Position.2,Ending Position.2,Non-peptidic Antigen IRI,Antigen Source Molecule Name,Antigen Source Molecule IRI,Protein Parent Name.1,Protein Parent IRI.1,Antigen Organism Name,Antigen Organism IRI,Organism Species Name,Organism Species IRI,Assay Comments
170410,http://www.iedb.org/assay/1841860,http://www.iedb.org/reference/1022232,Literature,21281984.0,Yan Li; Yan Zhu; Lin Zhou; Yimin Fang; Lirong ...,Immunobiology,2011,Use of HLA-DR*08032/E7 and HLA-DR*0818/E7 tetr...,NaN,http://www.iedb.org/epitope/145826,Linear peptide,IHSLLDEGKQSLTKL,25.0,39.0,NaN,6 kDa early secretory antigenic target,http://www.ncbi.nlm.nih.gov/protein/P0A564.2,6 kDa early secretory antigenic target,http://www.uniprot.org/uniprot/P9WNK7,Mycobacterium tuberculosis,http://purl.obolibrary.org/obo/NCBITaxon_1773,Mycobacterium tuberculosis,http://purl.obolibrary.org/obo/NCBITaxon_1773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T cell CD4+,http://purl.obolibrary.org/obo/CL_0000624,Direct Ex Vivo,NaN,NaN,NaN,Blood,http://purl.obolibrary.org/obo/UBERON_0000178,PBMC,http://purl.obolibrary.org/obo/CL_2000001,Direct Ex Vivo,HLA-DRB1*08:03,II,http://purl.obolibrary.org/obo/MRO_0001304,Cited reference,Epitope,Linear peptide,IHSLLDEGKQSLTKL,25.0,39.0,NaN,6 kDa early secretory antigenic target,http://www.ncbi.nlm.nih.gov/protein/P0A564.2,6 kDa early secretory antigenic target,http://www.uniprot.org/uniprot/P9WNK7,Mycobacterium tuberculosis,http://purl.obolibrary.org/obo/NCBITaxon_1773,Mycobacterium tuberculosis,http://purl.obolibrary.org/obo/NCBITaxon_1773,In situ detection of epitope/MHC II tetramer-b...
1150,http://www.iedb.org/assay/7903,http://www.iedb.org/reference/764,Literature,12853165.0,Boosbun Chua-Intra; Sukanya Wattanapokayakit; ...,Immunol Lett,2003,T-cell recognition of peptides from the Mycoba...,NaN,http://www.iedb.org/epitope/14086,Linear peptide,ERVAVKAEAGAGTEAPL,60.0,76.0,NaN,Major membrane protein I,http://www.ncbi.nlm.nih.gov/protein/P46841.2,Major membrane protein I,http://www.uniprot.org/uniprot/P46841,Mycobacterium leprae,http://purl.obolibrary.org/obo/NCBITaxon_1769,Mycobacterium leprae,http://purl.obolibrary.org/obo/NCBITaxon_1769,5 mismatches with M. Avium homolog region: R61...,NaN,NaN,NaN,NaN,NaN,NaN,...,PBMC,http://purl.obolibrary.org/obo/CL_2000001,Direct Ex Vivo,NaN,NaN,NaN,Blood,http://purl.obolibrary.org/obo/UBERON_0000178,PBMC,http://purl.obolibrary.org/obo/CL_2000001,Direct Ex Vivo,HLA class II,II,http://purl.obolibrary.org/obo/MRO_0001455,T cell assay -Biological process measured,Epitope,Linear peptide,ERVAVKAEAGAGTEAPL,60.0,76.0,NaN,Major membrane protein I,http://www.ncbi.nlm.nih.gov/protein/P46841.2,Major membrane protein I,http://www.uniprot.org/uniprot/P46841,Mycobacterium leprae,http://purl.obolibrary.org/obo/NCBITaxon_1769,Mycobacterium leprae,http://purl.obolibrary.org/obo/NCBITaxon_1769,Subjects with SI (cpm with antigen / cpm witho...
1153,http://www.iedb.org/assay/7909,http://www.iedb.org/reference/764,Literature,12853165.0,Boosbun Chua-Intra; Sukanya Wattanapokayakit; ...,Immunol Lett,2003,T-cell recognition of peptides from the Mycoba...,NaN,http://www.iedb.org/epitope/22776,Linear peptide,GTNQASDRTYLVTLYTSLVV,267.0,286.0,NaN,major membrane protein I,http://www.ncbi.nlm.nih.gov/protein/AAB60193

In [11]:
use_epitopes_df.Title.value_counts()

Memory T cells in latent Mycobacterium tuberculosis infection are directed against three antigenic islands and largely contained in a CXCR3+CCR6+ Th1 subset.                                     121
Comprehensive analysis of T cell immunodominance and immunoprevalence of SARS-CoV-2 epitopes in COVID-19 cases                                                                                    106
Neuroinvasive West Nile Infection Elicits Elevated and Atypically Polarized T Cell Responses That Promote a Pathogenic Outcome.                                                                   104
Integrated computational prediction and experimental validation identifies promiscuous T cell epitopes in the proteome of .                                                                        97
Characterization of Magnitude and Antigen Specificity of HLA-DP, DQ, and DRB3/4/5 Restricted DENV-Specific CD4+ T Cell Responses.                                                                  92
          

In [12]:
epitopes_df.loc[epitopes_df["Process Type"] == "Occurrence of infectious disease"].Title.value_counts().head(20)

Identification of MHC class II T cell Epitopes Derived from Mycobacterium tuberculosis                                                                           20563
Immune evasion versus recovery after acute hepatitis C virus infection from a shared source.                                                                      2407
Diversity in CD8(+) T cell function and epitope breadth among persons with genital herpes.                                                                        2251
Characterization of Magnitude and Antigen Specificity of HLA-DP, DQ, and DRB3/4/5 Restricted DENV-Specific CD4+ T Cell Responses.                                 1442
Differential antigenic hierarchy associated with spontaneous recovery from hepatitis C virus infection: implications for vaccine design.                          1241
Comprehensive analysis of T cell immunodominance and immunoprevalence of SARS-CoV-2 epitopes in COVID-19 cases                                                    123

In [13]:
epitopes_df["Cell Culture Conditions"].value_counts()

Direct Ex Vivo                                258469
Short Term Restimulated                        85886
Cell Line / Clone                              33747
Cell Line / Clone (Hybridoma)                   2545
In Vivo                                          977
Engineered                                       702
Blast Activated                                  170
Other / Unknown                                   49
Cell Line / Clone (EBV transformed, B-LCL)         1
Name: Cell Culture Conditions, dtype: int64

In [14]:
import json

antigens_df = pandas.read_csv(
    get_path("data_antigens", "epitope_to_sequence.csv.bz2"),
    converters={"other_antigen_names": json.loads})
antigens_df["uniprot_id"] = antigens_df.antigen_name.str.split("|").str.get(1)
antigens_df["id"] = antigens_df.antigen_name.str.split().str.get(0)
antigens_df
                              

,peptide,antigen_name,antigen_sequence,other_antigen_names,uniprot_id,id
0,KLEDLERDL,tr|Q9E927|Q9E927_HDV Large delta antigen OS=He...,MSRSESKRNRDGREGILEQWVNGRKKLEDLERDLRKIKKKIKKLED...,[tr|Q9E926|Q9E926_HDV Small delta antigen OS=H...,Q9E927,tr|Q9E927|Q9E927_HDV
1,IEQQADNMITEMLQK,tr|F1D8K8|F1D8K8_LASSJ Pre-glycoprotein polypr...,MGQIVTFFQEVPHVIEEVMNIVLIALSVLAVLKGLYNFATCGLVGL...,[],F1D8K8,tr|F1D8K8|F1D8K8_LASSJ
2,DLMGYIPLV,sp|Q81258|POLG_HCVNZ Genome polyprotein OS=Hep...,MSTLPKPQRKTKRNTIRRPQDVKFPGGGQIVGGVYVLPRRGPRLGV...,[sp|Q81495|POLG_HCVK3 Genome polyprotein OS=He...,Q81258,sp|Q81258|POLG_HCVNZ
3,QTVTSTPVQGR,tr|A0A076JQ90|A0A076JQ90_HCMV Extended tegumen...,MSLQFIGLQRRDVVALVNFLRHLTQKPDVDLEAHPKILKKCGEKRL...,[tr|A0A0G2TTT7|A0A0G2TTT7_HCMV Tegument protei...,A0A076JQ90,tr|A0A076JQ90|A0A076JQ90_HCMV
4,TTVYPPSSTAK,tr|A0A076JQ90|A0A076JQ90_HCMV Extended tegumen...,MSLQFIGLQRRDVVALVNFLRHLTQKPDVDLEAHPKILKKCGEKRL...,[tr|A0A0G2U3E2|A0A0G2U3E2_HCMV Tegument protei...,A0A076JQ90,tr|A0A076JQ90|A0A076JQ90_HCMV
...,...,...,...,...,...,...
58076,SPERKMLPC,tr|Q65613|Q65613_POVBK Capsid protein VP1 OS=B...,MAPTKRKGECPGAAPKKPKEPVQVPKLLIKGGVEVLEVKTGVDAIT...,[tr|B9X1W0|B9X1W0_POVBK Major capsid protein V...,Q65613,tr|Q65613|Q65613_POVBK
58077,TVATSRTLSY,tr|A0A6V7AL93|A0A6V7AL93_SARS2 Membrane glycop...,MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFL...,[tr|A7J8L8|A7J8L8_SARS Membrane protein OS=Sev...,A0A6V7AL93,tr|A0A6V7AL93|A0A6V7AL93_SARS2
58078,ILMTARTVY,tr|A0A6G7SLW8|A0A6G7SLW8_SARS2 2'-O-methyltran...,MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...,[tr|A0A6C0QEN3|A0A6C0QEN3_SARS2 2'-O-methyltra...,A0A6G7SLW8,tr|A0A6G7SLW8|A0A6G7SLW8_SARS2
58079,RTATKAYNV,tr|A0A6G7S6L7|A0A6G7S6L7_SARS2 Nucleoprotein O...,MSDNGPQNQRNAPRITFGGPSDSTGSNQNGEXSGARSKQRRPQGLP...,[tr|A0A6H0C6H8|A0A6H0C6H8_SARS2 Nucleoprotein ...,A0A6G7S6L7,tr|A0A6G7S6L7|A0A6G7S6L7_SARS2


In [15]:
antigens_merged_df = antigens_df.merge(use_epitopes_df, left_on="peptide", right_on="Description")
antigens_merged_df

,peptide,antigen_name,antigen_sequence,other_antigen_names,uniprot_id,id,Assay IRI,Reference IRI,Type,PubMed ID,Authors,Journal,Date,Title,Submission ID,Epitope IRI,Object Type,Description,Starting Position,Ending Position,Non-peptidic epitope IRI,Antigen Name,Antigen IRI,Parent Protein,Parent Protein IRI,Organism Name,Organism IRI,Parent Species,Parent Species IRI,Epitope Comments,...,Cell Type,Effector Cell Type IRI,Cell Culture Conditions,Assayed TCR Molecule Name,TCR IRI,PDB ID,Cell Tissue Type.1,APC Cell Tissue Type IRI,Cell Type.1,APC Cell Type IRI,Cell Culture Conditions.1,Allele Name,Class,Allele IRI,Allele Evidence Code,Antigen Epitope Relation,Antigen Object Type,Antigen Description,Starting Position.2,Ending Position.2,Non-peptidic Antigen IRI,Antigen Source Molecule Name,Antigen Source Molecule IRI,Protein Parent Name.1,Protein Parent IRI.1,Antigen Organism Name,Antigen Organism IRI,Organism Species Name,Organism Species IRI,Assay Comments
0,KLVALGINAV,sp|P26664|POLG_HCV1 Genome polyprotein OS=Hepa...,MSTNPKPQKKNKRNTNRRPQDVKFPGGGQIVGGVYLLPRRGPRLGV...,[sp|P27958|POLG_HCV77 Genome polyprotein OS=He...,P26664,sp|P26664|POLG_HCV1,http://www.iedb.org/assay/1467426,http://www.iedb.org/reference/1006655,Literature,17462079.0,Silvia Della Bella; Andrea Crosignani; Antonio...,Immunology,2007,Decrease and dysfunction of dendritic cells co...,NaN,http://www.iedb.org/epitope/32208,Linear peptide,KLVALGINAV,1406.0,1415.0,NaN,Genome polyprotein,http://www.ncbi.nlm.nih.gov/protein/P26664.3,Genome polyprotein,http://www.uniprot.org/uniprot/P27958,Hepatitis C virus subtype 1b,http://purl.obolibrary.org/obo/NCBITaxon_31647,Hepacivirus C,http://purl.obolibrary.org/obo/NCBITaxon_11103,The exact sequences of the epitopes described ...,...,T cell CD4+,http://purl.obolibrary.org/obo/CL_0000624,Direct Ex Vivo,NaN,NaN,NaN,Blood,http://purl.obolibrary.org/obo/UBERON_0000178,PBMC,http://purl.obolibrary.org/obo/CL_2000001,Direct Ex Vivo,HLA class II,II,http://purl.obolibrary.org/obo/MRO_0001455,T cell assay -T cell subset identification,Epitope,Linear peptide,KLVALGINAV,1406.0,1415.0,NaN,Genome polyprotein,http://www.ncbi.nlm.nih.gov/protein/P26664.3,Genome polyprotein,http://www.uniprot.org/uniprot/P27958,Hepatitis C virus subtype 1b,http://purl.obolibrary.org/obo/NCBITaxon_31647,Hepacivirus C,http://purl.obolibrary.org/obo/NCBITaxon_11103,BrdU incorporation was measured using flow cyt...
1,DRAHYNIVTFCCKCD,tr|D7RGM8|D7RGM8_HPV16 Protein E7 OS=Human pap...,MHGDTPTLHEYMLDLQPETTDLYCYEQSPFSSEEEDEIDGPAGQAE...,[tr|B9UPB6|B9UPB6_HPV16 Protein E7 OS=Human pa...,D7RGM8,tr|D7RGM8|D7RGM8_HPV16,http://www.iedb.org/assay/1657910,http://www.iedb.org/reference/1015785,Literature,16112023.0,Dominic E Warrino; Walter C Olson; Meera I Sca...,Hum Immunol,2005,Human papillomavirus L1L2-E7 virus-like partic...,NaN,http://www.iedb.org/epitope/9947,Linear peptide,DRAHYNIVTFCCKCD,48.0,62.0,NaN,E7 protein,http://www.ncbi.nlm.nih.gov/protein/ABL96585.1,Protein E7,http://www.uniprot.org/uniprot/P03129,Human papillomavirus type 16,http://purl.obolibrary.org/obo/NCBITaxon_333760,Alphapapillomavirus 9,http://purl.obolibrary.org/obo/NCBITaxon_337041,NaN,...,T cell CD4+,http://purl.obolibrary.org/obo/CL_0000624,Direct Ex Vivo,NaN,NaN,NaN,Blood,http://purl.obolibrary.org/obo/UBERON_0000178,Dendritic cell,http://purl.obolibrary.org/obo/CL_0000451,Direct Ex Vivo,HLA-DR,II,http://purl.obolibrary.org/obo/MRO_0000013,Cited reference,Epitope,Linear peptide,DRAHYNIVTFCCKCD,48.0,62.0,NaN,E7 protein,http://www.ncbi.nlm.nih.gov/protein/ABL96585.1,Protein E7,http://www.uniprot.org/uniprot/P03129,Human papillomavirus type 16,http://purl.obolibrary.org/obo/NCBITaxon_333760,Alphapapillomavirus 9,http://purl.obolibrary.org/obo/NCBITaxon_337041,NaN
2,MHGDTPTLHEYM,tr|D7RGM8|D7RGM8_HPV16 Protein E7 OS=Human pap...,MHGDTPTLHEYMLDLQPETTDLYCYEQSPFSSEEEDEIDGPAGQAE...,[tr|B9UPB6|B9UPB6_HPV16 Protein E7 OS=Human pa...,D7RGM8,tr|D7RGM8|D7RGM8_HPV16,http://www.iedb.org/assay/1657891,http://www.iedb.

In [16]:
case_encoded_df = []
for seq, sub_df in antigens_merged_df.groupby("antigen_sequence"):
    (name,) = sub_df.antigen_name.unique()
    for peptide in sub_df.peptide:
        seq = seq.replace(peptide.upper(), peptide.lower())
    case_encoded_df.append((name, seq, sub_df.peptide.nunique()))
case_encoded_df = pandas.DataFrame(case_encoded_df, columns=["name", "seq", "epitopes"])
case_encoded_df["epitope_fraction"] = case_encoded_df.seq.map(lambda s: numpy.mean([c == c.lower() for c in s]))
case_encoded_df["input"] = case_encoded_df.seq.map(lambda s: re.sub(r"[UZOB]", "X", s.upper())).map(list)
case_encoded_df["output"] = case_encoded_df.seq.map(lambda s: [int(c == c.lower()) for c in s])

case_encoded_df = case_encoded_df.sort_values("epitopes")
case_encoded_df

,name,seq,epitopes,epitope_fraction,input,output
0,tr|D3TK66|D3TK66_HBV Protein X OS=Hepatitis B ...,MAARVCCQLDPARDVLCLRPVGAESRGRPVSGPFGALPSPSSSAVP...,1,0.133333,"[M, A, A, R, V, C, C, Q, L, D, P, A, R, D, V, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
89,tr|U5QB44|U5QB44_HBV Capsid protein OS=Hepatit...,MQLFHLCLKNLCSCPNCYKTHPCLGGLWGMDIDPYKEFGasvells...,1,0.091743,"[M, Q, L, F, H, L, C, L, K, N, L, C, S, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88,tr|D0EEC4|D0EEC4_HBV Capsid protein OS=Hepatit...,MQLFHLCLIISCTCPTVQASKLCLGWLWGMDIDPYKEFGATVELLS...,1,0.070093,"[M, Q, L, F, H, L, C, L, I, I, S, C, T, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
87,tr|D0EEP3|D0EEP3_HBV Capsid protein OS=Hepatit...,MQLFHLCLIISCTCPTVQASKLCLGWLWGMDIDPYKEFGATVELLS...,1,0.093458,"[M, Q, L, F, H, L, C, L, I, I, S, C, T, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
86,tr|L7X6M1|L7X6M1_HBV Capsid protein OS=Hepatit...,MQLFHLCLIISCTCPTVQASKLCLGWLWGMDIDLYKEFGATVELLS...,1,0.093458,"[M, Q, L, F, H, L, C, L, I, I, S, C, T, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
51,tr|A0A1D9C0X2|A0A1D9C0X2_ZIKV Core protein OS=...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGhgpirm...,36,0.102249,"[M, K, N, P, K, K, K, S, G, G, F, R, I, V, N, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
108,tr|A0A0H3V8V9|A0A0H3V8V9_WNV Core protein OS=W...,MSKKPGGPGKNRAVNMLKRGMPRGLSLIGLKRAMLSLIDGKGPIRF...,37,0.131625,"[M, S, K, K, P, G, G, P, G, K, N, R, A, V, N, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
32,tr|A4GEE2|A4GEE2_HHV3 Envelope glycoprotein E ...,MFYEALKAELVYTRAVHGFRPRANCVVLSDYIPRVACNMGTVNKPV...,41,0.459909,"[M, F, Y, E, A, L, K, A, E, L, V, Y, T, R, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
110,tr|E9NMD6|E9NMD6_WNV Core protein OS=West Nile...,MSKKPGGPGKSRAVNmlkrgmprvlsliglkraMLSLIDGKGPIRF...,51,0.183804,"[M, S, K, K, P, G, G, P, G, K, S, R, A, V, N, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
train_df = case_encoded_df.sample(frac=0.8)
val_df = case_encoded_df.loc[~case_encoded_df.index.isin(train_df.index)]
train_df

,name,seq,epitopes,epitope_fraction,input,output
115,tr|B2VMQ5|B2VMQ5_I68A5 Matrix protein 1 OS=Inf...,MSLLTEVETYVLSIVPSGPLKAEIAQRLEDVFAGKNTDLEALMEWL...,1,0.043651,"[M, S, L, L, T, E, V, E, T, Y, V, L, S, I, V, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
18,tr|E9N3C0|E9N3C0_HEV Methyltransferase OS=Hepa...,MEAHQFIKAPGITTAIEQAALAAANSALANAVVVRPFLSRLQTEIL...,1,0.008499,"[M, E, A, H, Q, F, I, K, A, P, G, I, T, T, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
141,sp|Q00269|POLG_HCVJT Genome polyprotein OS=Hep...,MSTNPKPQRKTKRNTYRRPQDVKFPGGGQIVGGVYVLPRRGPTLGV...,1,0.007973,"[M, S, T, N, P, K, P, Q, R, K, T, K, R, N, T, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,sp|K9N4V7|NCAP_MERS1 Nucleoprotein OS=Middle E...,MASPAAPRAVSFADNNDITNTNLSRGRGRNPKPRAAPNNTVSWYTG...,3,0.145985,"[M, A, S, P, A, A, P, R, A, V, S, F, A, D, N, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
41,tr|A0A663DJA2|A0A663DJA2_SARS2 ORF10 protein O...,MGYinvfafpftiyslllCRMNSRNYIAQVDVVNFNLT,1,0.394737,"[M, G, Y, I, N, V, F, A, F, P, F, T, I, Y, S, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...
63,sp|Q5UB51|POLG_DEN3I Genome polyprotein OS=Den...,MNNQRKKTGKPSINMLKRVRNRVSTGSQLAKRFSRGLLNGQGPMKL...,14,0.058997,"[M, N, N, Q, R, K, K, T, G, K, P, S, I, N, M, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
22,tr|A0A0C7SYE0|A0A0C7SYE0_EBVG EBNA-3C OS=Epste...,MESFEGQGDSRQSPDNERGDNVQTTGEHDQDPGPGPPSSGASERLV...,1,0.014735,"[M, E, S, F, E, G, Q, G, D, S, R, Q, S, P, D, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88,tr|D0EEC4|D0EEC4_HBV Capsid protein OS=Hepatit...,MQLFHLCLIISCTCPTVQASKLCLGWLWGMDIDPYKEFGATVELLS...,1,0.070093,"[M, Q, L, F, H, L, C, L, I, I, S, C, T, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
45,tr|A0A384KEH1|A0A384KEH1_HPV16 Protein E6 OS=H...,MHQKRTAMFQDPQERprklpqlctelqttiHDIILECVYCKQQLLR...,3,0.094937,"[M, H, Q, K, R, T, A, M, F, Q, D, P, Q, E, R, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [18]:
val_df

,name,seq,epitopes,epitope_fraction,input,output
85,tr|C7AYU5|C7AYU5_HBV Capsid protein OS=Hepatit...,MQLFHLCLIISCTCPTFQASKLCLGWLWGMDIDPYKEFGATVELLS...,1,0.093458,"[M, Q, L, F, H, L, C, L, I, I, S, C, T, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
80,tr|Q4W6F7|Q4W6F7_HBV Capsid protein OS=Hepatit...,MQLFHLCLIISCSCPTVQASKLCLGWLWGMDIDPYKEFGATVELLS...,1,0.094340,"[M, Q, L, F, H, L, C, L, I, I, S, C, S, C, P, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
71,tr|A0A0C7TKT1|A0A0C7TKT1_EBVG EBNA-2 OS=Epstei...,MPTFYLALHGGQTYHLIVDTDSRGNPSLSVIPSNPYQEQLSDTPLI...,1,0.040984,"[M, P, T, F, Y, L, A, L, H, G, G, Q, T, Y, H, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
97,tr|A0A6H0C6H8|A0A6H0C6H8_SARS2 Nucleoprotein O...,MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLP...,1,0.035800,"[M, S, D, N, G, P, Q, N, Q, R, N, A, P, R, I, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
144,tr|J3UEA1|J3UEA1_HPV16 Probable protein E5 OS=...,MTNLDTASTTLLACFLLCFCVLLCVCLLIRPLllsvstytslillv...,1,0.180723,"[M, T, N, L, D, T, A, S, T, T, L, L, A, C, F, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
116,sp|K9N7A1|VME1_MERS1 Membrane protein OS=Middl...,MSNMTQLTEAQIIAIIKDWNFAWSLIFLLITIVLQYGYPSRSMTVY...,1,0.091324,"[M, S, N, M, T, Q, L, T, E, A, Q, I, I, A, I, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
109,tr|F4YXG0|F4YXG0_WNV Core protein OS=West Nile...,MSKKPGGPGKNRAVNMLKRGMPRGLSLIGLKRAMLSLIDGKGPIRF...,1,0.005242,"[M, S, K, K, P, G, G, P, G, K, N, R, A, V, N, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
103,tr|Q0H7F2|Q0H7F2_WNV Core protein OS=West Nile...,MSKKPGGPGKNRAVNMLKRGMPRGLSLIGLKRAMLSLIDGKGPIRF...,1,0.005242,"[M, S, K, K, P, G, G, P, G, K, N, R, A, V, N, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
102,sp|Q9PZT0|CAPSD_PAVHV Minor capsid protein VP1...,MSKESGKWWESDDKFAKAVYQQFVEFYEKVTGTDLELIQILKDHYN...,1,0.025608,"[M, S, K, E, S, G, K, W, W, E, S, D, D, K, F, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
101,sp|Q98803|POLG_YEFVI Genome polyprotein OS=Yel...,MSGRKAQGKTLGVNMVRRGVRSLSNKIKQKTKQIGNRPGPSRGVQG...,1,0.004398,"[M, S, G, R, K, A, Q, G, K, T, L, G, V, N, M, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
#case_encoded_df.iloc[-1].output

In [20]:
#model_name = 'Rostlab/prot_bert_bfd'
model_name = 'Rostlab/prot_bert'
seq_tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=False)
seq_tokenizer

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_bert/resolve/main/vocab.txt HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_bert/resolve/main/tokenizer.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_bert/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_bert/resolve/main/special_tokens_map.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_be

PreTrainedTokenizerFast(name_or_path='Rostlab/prot_bert', vocab_size=30, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [21]:
train_seq_encodings = seq_tokenizer(train_df.input.tolist(), is_split_into_words=True, return_offsets_mapping=True, truncation=True, padding=True)
len(train_seq_encodings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


4

In [22]:
val_seq_encodings = seq_tokenizer(val_df.input.tolist(), is_split_into_words=True, return_offsets_mapping=True, truncation=True, padding=True)
len(val_seq_encodings)

4

In [23]:
#train_seq_encodings['input_ids'][0]

In [24]:
#!pip install --upgrade ipywidgets

In [25]:
train_labels = train_df.output.tolist()
len(train_labels)

120

In [26]:
val_labels = val_df.output.tolist()
len(val_labels)

30

In [27]:
unique_tags = set(tag for doc in train_labels for tag in doc)
unique_tags  = sorted(list(unique_tags))  # make the order of the labels unchanged
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [28]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels_encodings = encode_tags(train_labels, train_seq_encodings)
val_labels_encodings = encode_tags(val_labels, val_seq_encodings)

len(train_labels_encodings), len(val_labels_encodings)

(120, 30)

In [29]:
train_labels_encodings[0][:10]

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
if "offset_mapping" in train_seq_encodings.keys():
    train_seq_encodings.pop("offset_mapping")

if "offset_mapping" in val_seq_encodings.keys():
    val_seq_encodings.pop("offset_mapping")

train_dataset = MyDataset(train_seq_encodings, train_labels_encodings)
val_dataset = MyDataset(val_seq_encodings, val_labels_encodings)
val_dataset

In [31]:
def align_predictions(predictions: np.ndarray, label_ids: np.ndarray):
        preds = np.argmax(predictions, axis=2)

        batch_size, seq_len = preds.shape

        out_label_list = [[] for _ in range(batch_size)]
        preds_list = [[] for _ in range(batch_size)]

        for i in range(batch_size):
            for j in range(seq_len):
                if label_ids[i, j] != torch.nn.CrossEntropyLoss().ignore_index:
                    out_label_list[i].append(id2tag[label_ids[i][j]])
                    preds_list[i].append(id2tag[preds[i][j]])

        return preds_list, out_label_list

def compute_metrics(p: EvalPrediction):
    preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
    return {
        "accuracy": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

In [32]:
MODEL = None

def model_init(fix_weights=200):
    global MODEL
    model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                         num_labels=len(unique_tags),
                                                         id2label=id2tag,
                                                         label2id=tag2id,
                                                         gradient_checkpointing=False)
    if fix_weights:
        for name, param in list(model.named_parameters())[:fix_weights]:
            print("Fixing", name)
            param.requires_grad = False

    MODEL = model
    return model


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    learning_rate=3e-05,             # learning rate
    weight_decay=0.0,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=32,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-SS3",      # experiment name
    seed=3,                         # Seed for experiment reproducibility
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model_init=model_init,                # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,    # evaluation metrics
)

trainer.train()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_bert/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /Rostlab/prot_bert/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another ar

RuntimeError: ignored

In [ ]:
len(list(MODEL.bert.named_parameters()))

In [ ]:
MODEL